## Generate Trading Signals for Cointegrated Pairs

In [19]:

import os
import numpy as np
import pandas as pd
from IPython.display import display

In [20]:
"""
Generate Trading Signals for Cointegrated Pairs

For each pair (i,j) in cointegrated_pairs.csv, we:
  1. Compute the spread: S_t = price_i_t - beta * price_j_t
  2. Compute rolling mean μ_t and rolling std σ_t of the spread over the past N days.
  3. Calculate z-score: z_t = (S_t - μ_{t-1}) / σ_{t-1}
  4. Generate position_t:
       - +1 (long spread)   if z_t < -entry_threshold
       - -1 (short spread)  if z_t > +entry_threshold
       -  0 (flat)          if |z_t| < exit_threshold
       - otherwise carry forward yesterday’s position
  5. Output a DataFrame of positions (dates × pairs).

Industry notes:
  • N = 60 days is common for mid‐term mean reversion.
  • Entry thresholds ~1.5σ, exit ~0.5σ are standard.
  • z-score is simply “how far from normal” in units of volatility.

"""


#1) SETTINGS 

DATA_DIR        = "Data"
OUTPUT_DIR      = "Output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

PRICE_CSV       = os.path.join(DATA_DIR, "sp500_prices_clean.csv")
PAIRS_CSV       = os.path.join(OUTPUT_DIR, "cointegrated_pairs.csv")
SIGNALS_CSV     = os.path.join(OUTPUT_DIR, "signals.csv")

# Rolling window length for mean/std (in trading days)
ROLL_WINDOW     = 22

# Entry/exit thresholds (in z-score units)
ENTRY_THRESH    = 1.5
EXIT_THRESH     = 0.5

#2) LOAD DATA 

# Load cleaned prices and cointegrated pairs
prices = pd.read_csv(PRICE_CSV, index_col=0, parse_dates=True)
pairs  = pd.read_csv(PAIRS_CSV)

# Create a date index for signals
dates = prices.index[ROLL_WINDOW:]  # start after window to avoid NaNs

#3) PREPARE A PLACEHOLDER FOR SIGNALS 

# We’ll name each column "i_j" for pair (i,j)
pair_names = [f"{row.ticker_i}_{row.ticker_j}" for _, row in pairs.iterrows()]

# DataFrame to hold the daily position for each pair
signals_df = pd.DataFrame(
    data=0,                  # initialize all positions to flat
    index=dates,
    columns=pair_names
)

#4) LOOP OVER EACH PAIR AND GENERATE SIGNALS 

for idx, row in pairs.iterrows():
    ti, tj, beta = row.ticker_i, row.ticker_j, row.beta
    col_name = f"{ti}_{tj}"

    # Compute spread series S_t
    spread = prices[ti] - beta * prices[tj]

    # Rolling statistics: use .shift(1) for look‐ahead safety
    rolling_mean  = spread.rolling(window=ROLL_WINDOW).mean().shift(1)
    rolling_std   = spread.rolling(window=ROLL_WINDOW).std().shift(1)

    # Compute z-score
    z_score = (spread - rolling_mean) / rolling_std

    # Initialize a position series (+1, -1, or 0)
    pos = pd.Series(index=spread.index, dtype=int)
    pos.iloc[:ROLL_WINDOW] = 0  # flat until we have enough history

    # Iterate over dates to apply rules
    for t in range(ROLL_WINDOW, len(spread)):
        date = spread.index[t]
        z = z_score.iloc[t]

        # Entry signals
        if z < -ENTRY_THRESH:
            pos.iloc[t] = +1
        elif z > +ENTRY_THRESH:
            pos.iloc[t] = -1
        # Exit signal
        elif abs(z) < EXIT_THRESH:
            pos.iloc[t] = 0
        # Otherwise, carry forward yesterday's position
        else:
            pos.iloc[t] = pos.iloc[t-1]

    # Trim to our common dates and assign to signals_df
    signals_df[col_name] = pos.loc[dates]

# 5) SAVE SIGNALS 

signals_df.to_csv(SIGNALS_CSV)
print(f"Generated signals for {len(pair_names)} pairs.")
print(f"Signals saved to '{SIGNALS_CSV}'.")


Generated signals for 144 pairs.
Signals saved to 'Output/signals.csv'.


In [21]:
# 6) LOAD AND DISPLAY SIGNALS
signals = pd.read_csv("Output/signals.csv", index_col=0, parse_dates=True)


# Display the first few rows and summary of positions
display(signals.head())
display(signals.apply(pd.Series.value_counts))


,A_DHR,A_IQV,A_MTD,A_TMO,ABBV_COR,ABT_STE,ABT_ZTS,ACN_PAYX,ADBE_ANSS,ADBE_WDAY,...,RL_TPR,RMD_STE,RMD_WST,ROL_VRSK,RVTY_TECH,STZ_SYY,TFC_USB,TMO_ZTS,TTWO_WDAY,VLO_XOM
Date,,,,,,,,,,,,,,,,,,,,,
2015-02-04,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,-1.0,0.0,1.0,0.0,1.0,0.0
2015-02-05,0.0,0.0,0.0,0.0,1.0,0.0,-1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,-1.0,0.0,1.0,0.0,1.0,0.0
2015-02-06,0.0,0.0,0.0,0.0,1.0,0.0,-1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,1.0,0.0
2015-02-09,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,1.0,0.0
2015-02-10,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,1.0,1.0,1.0,...,1.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,1.0,0.0


,A_DHR,A_IQV,A_MTD,A_TMO,ABBV_COR,ABT_STE,ABT_ZTS,ACN_PAYX,ADBE_ANSS,ADBE_WDAY,...,RL_TPR,RMD_STE,RMD_WST,ROL_VRSK,RVTY_TECH,STZ_SYY,TFC_USB,TMO_ZTS,TTWO_WDAY,VLO_XOM
-1.0,782,779,662,749,886,769,707,834,804,819,...,751,794,807,844,783,757,809,679,799,913
0.0,952,974,1053,1008,849,967,1012,924,948,942,...,1034,1025,930,853,990,1078,930,1087,1035,905
1.0,759,740,778,736,758,757,774,735,741,732,...,708,674,756,796,720,658,754,727,659,675
